### 1.	Реализуйте базовый частотный метод по Шерлоку Холмсу:
* подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
* возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
* расшифруйте их таким частотным методом.

In [1]:
import re
import random
import numpy as np
from collections import Counter
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

random.seed(42)

In [2]:
PATH_WAR = "/kaggle/input/tolstoy-novels/WarAndPeace.txt"
PATH_ANNA = "/kaggle/input/tolstoy-novels/AnnaKarenina.txt"

UNK = '&' 

In [3]:
with open(PATH_WAR, 'r') as f_ru:
     wap = f_ru.read()

In [4]:
def preprocess_text(text):
    prep_text = re.sub("[^а-я ]", " ", text.lower())
    return re.sub(r'\s+', ' ',prep_text)

wap = preprocess_text(wap)

In [5]:
def train_test_split(text, n):
    train_text, test_text = text[:n], text[n:]
    return train_text, test_text

In [6]:
n = -10000

train_wap, test_wap = train_test_split(wap, n)

In [7]:
char_cnt = Counter(train_wap)
chars = list(char_cnt.keys())
chars_encoded = chars.copy()
np.random.shuffle(chars_encoded)
print(chars)
print()
print(chars_encoded)
encode_mapping = dict(zip(chars, chars_encoded))
decode_mapping = {v: k for k, v in encode_mapping.items()} # обратный маппинг для оценки

[' ', 'в', 'о', 'й', 'н', 'а', 'и', 'м', 'р', 'с', 'ы', 'з', 'е', 'т', 'л', 'ь', 'к', 'ч', 'г', 'д', 'у', 'п', 'я', 'ж', 'б', 'щ', 'ф', 'э', 'х', 'ю', 'ш', 'ц', 'ъ']

['е', 'в', 'ы', 'п', 'э', 'ш', 'я', ' ', 'р', 'ю', 'у', 'м', 'ь', 'б', 'с', 'ъ', 'к', 'х', 'ц', 'з', 'о', 'д', 'ф', 'н', 'л', 'щ', 'а', 'т', 'ч', 'г', 'и', 'й', 'ж']


In [8]:
def encode_text(text, mapping):
    text_encoded = ''
    for char in text:
        text_encoded += mapping.get(char, UNK)
    return text_encoded

test_encoded = encode_text(test_wap, encode_mapping)

In [9]:
test_encoded[:500]

'звяэосшюъеюншсшюъебрыэосшюъеэьюкысъкыеишцывеяеыдфбъеыюбшэывясшюъедрыпбяетбяеюбыеишцывеяеэшвьрэыьеюдшюьэедрыюбыфбъеьщьезвье яэобуеяедыцялеэшвьрэыьезо шсекшнзупезысычывеюбыфвияпевеюьрьзяэьебысдуервшэосюфекекршгедсыбяэуеюлявеюеэыцезвочеюысзшбеяеюльншсеэшеюкысъмкяпесьзедыкрувияпедрозеювыршхявшпемшкряхшсеыэедыздруцявшфедыесъзоекыбырупебрьщшседызеэя еювыршхявшпекряхшсеыэеэшеырозяьезьрнябесьзезьрншсеьцыеэыецэосюфеяебрьщшсеяеыхьвязэыелусыехбыеэьебысъкыедызеырозяь еясяебысдыпеэшрызшеэыедызеэя еызэя еыэею'

In [10]:
def decode(text, freq_mapping):
    char_cnt_encoded = Counter(text).most_common()
    freq_mapping = freq_mapping.most_common()
    mapping = {}
    for i, (char, _) in enumerate(char_cnt_encoded):
        mapping[char] = freq_mapping[i][0]
    text_decoded = ''
    for char in text:
        text_decoded += mapping.get(char, UNK)
    return text_decoded, mapping

test_decoded, freq_mapping = decode(test_encoded, char_cnt)

In [11]:
test_decoded[:500]

'двниптесз сшетесз лроиптесз иаскотзко жегов н оуялз ослеиовнтесз уройлн юлн сло жегов н иевариоа суесаи урослоялз аэа два мнипль н уогнб иевариоа дпмет кешдьй дотохов слоявжнй в сарадниа лотуь рвеиптся к крец утолниь сбнв с иог двпх сотдел н сбашет ие скотзыкнй тад уокрьвжнй урпд своречнвей ыекрнчет ои уодурьгнвея уо тздп колорьй лраэет уод инм своречнвей крнчет ои ие орпдна даршнл тад даршет аго ио гиптся н лраэет н очавндио бьто чло иа лотзко уод орпднам нтн лотуой иероде ио уод инм одинм ои с'

In [12]:
def get_accuracy(freq_mapping, decode_mapping):
    correctly_mapped_chars = sum(1 for char in freq_mapping.keys() if freq_mapping[char] == decode_mapping[char])
    total_chars = len(freq_mapping)
    return correctly_mapped_chars / total_chars

In [13]:
print(f"Точность: {get_accuracy(freq_mapping, decode_mapping)}")

Точность: 0.42424242424242425


**2.	Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:**
* подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
* проведите тестирование аналогично п.1, но при помощи биграмм (В качестве естественной метрики качества можно взять долю правильно расшифрованных букв или, если хочется совсем математически изощриться, расстояние между двумя перестановками, правильной и полученной из модели; но, честно говоря, в этом задании следить за численными метриками не так уж обязательно, будет и глазами всё видно).

In [14]:
def count_bigrams(text):
    bigrams = [text[i:i + 2] for i in range(0, len(text) - 1, 2)]
    bigram_counts = {}
    for bigram in bigrams:
        bigram_counts[bigram] = bigram_counts.get(bigram, 0) + 1
    return bigram_counts

bigram_cnt = count_bigrams(train_wap)

In [15]:
test_encoded[:500]

'звяэосшюъеюншсшюъебрыэосшюъеэьюкысъкыеишцывеяеыдфбъеыюбшэывясшюъедрыпбяетбяеюбыеишцывеяеэшвьрэыьеюдшюьэедрыюбыфбъеьщьезвье яэобуеяедыцялеэшвьрэыьезо шсекшнзупезысычывеюбыфвияпевеюьрьзяэьебысдуервшэосюфекекршгедсыбяэуеюлявеюеэыцезвочеюысзшбеяеюльншсеэшеюкысъмкяпесьзедыкрувияпедрозеювыршхявшпемшкряхшсеыэедыздруцявшфедыесъзоекыбырупебрьщшседызеэя еювыршхявшпекряхшсеыэеэшеырозяьезьрнябесьзезьрншсеьцыеэыецэосюфеяебрьщшсеяеыхьвязэыелусыехбыеэьебысъкыедызеырозяь еясяебысдыпеэшрызшеэыедызеэя еызэя еыэею'

In [16]:
def decode_text_by_bigrams(text, bigram_freq_mapping):
    bigrams = [text[i:i + 2] for i in range(0, len(text), 2)]
    
    mapping = {}
    for bigram, (symbol, _) in zip(bigrams, bigram_freq_mapping.items()):
        mapping[bigram] = symbol
    
    text_decoded = ''.join(mapping.get(bigram, '') for bigram in bigrams)
    
    return text_decoded

test_decoded = decode_text_by_bigrams(test_encoded, bigram_cnt)

In [17]:
test_decoded[:500]

'ждюжьи рьгирйе рьгжеф ьи рьгюсычсфа эпууъедблзыдшоьг ытябсюшбнвгкгптпрлзедлзштэпууъедблзуцшкжаьбшспямедшса ынпшоьгывйрждйркнздыжсгкгдаечлд ьддбсйрззяетгяюо пэьшсфер йшснпзамюжбдбмехмжююсбцсфюм эсъздбчнзтжнфбщкгвщкажсшсвздбьобсшежду шссфцлсблзвхстйелдтзычсфонуфжблбгмншнф бмюжбсакшшсыуьщмксъжбжмнфкдйелмдшншйчсытьсънзншгшлезсвлнпсыжбжеывйекгмглдфашсыуьщмксъжбнфкдйелмдшуцлмнвжюйршахр ягшдзьшддлвтгхнэпбсбкздбчнзлзжеывйесглмрпюшщ эппфвщккнплдйрнпумвлкгмглмнвжюднсгжжбцсфншжбуцптцллдэпншгмпшюнмгпшюнф шс'

In [18]:
def get_accuracy_(encoded_text, decoded_text, decode_mapping):
    mapping = {}
    for i in range(len(decoded_text)):
        if encoded_text[i] not in mapping:
            mapping[encoded_text[i]] = decoded_text[i]
    total_chars = len(mapping)
    correctly_mapped_chars = sum(1 for k in mapping.keys() if mapping[k] == decode_mapping[k])
    return correctly_mapped_chars / total_chars

In [19]:
print(f"Точность: {get_accuracy_(test_encoded, test_decoded, decode_mapping)}")

Точность: 0.030303030303030304


Декодировка текста с использованием биграмм показывает результаты похуже по сравнению с декодированием на отдельных символах. Возможная причина - в тестовом корпусе присутствует больше различных комбинаций биграмм, чем отдельных символов

### 3.	Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
* предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
* реализуйте и протестируйте его, убедитесь, что результаты улучшились.

### Метод обучения перестановки символов на основе MCMC-сэмплирования с использованием статистики биграмм

Общая идея: изначально случайным образом выбирается перестановка символов. На каждой итерации алгоритма генерируется кандидат в новое состояние путем обмена местами двух случайно выбранных символов в текущей перестановке. Затем вычисляются вероятности встретить биграмму в корпусе для текста, расшифрованного с использованием новой перестановки и текущей перестановки.

Вероятность принятия новой перестановки определяется сравнением произведений вероятностей биграмм для нового и текущего состояний. Если произведение вероятностей для нового состояния больше или равно произведения вероятностей для текущего состояния, новая перестановка принимается. В противном случае, новая перестановка принимается с вероятностью, пропорциональной отношению вероятностей.

Процесс запускается несколько раз. После завершения MCMC-сэмплирования выбирается перестановка символов с наибольшим логарифмом правдоподобия, представляющая оптимальноую декодировку

In [25]:
def get_bigrams(text):
    return [text[i:i + 2] for i in range(0, len(text) - 1)]

def calc_log_likelihood(decoded_text, train_freqs):
    total_bigrams = sum(train_freqs.values())
    log_likelihood = 0
    for i in range(len(decoded_text) - 1):
        bigram = decoded_text[i:i+2]
        log_likelihood += np.log((train_freqs[bigram] + 1) / (total_bigrams + 1))
    return log_likelihood

def random_swap_chars(chars_list):
    chars_list_local = chars_list.copy()
    ids_to_swap = np.random.choice(len(chars_list_local), size=2, replace=False)
    chars_list_local[ids_to_swap[0]], chars_list_local[ids_to_swap[1]] = chars_list_local[ids_to_swap[1]], chars_list_local[ids_to_swap[0]]
    return chars_list_local

def decode_text_by_mapping(text, mapping):
    text_decoded = ''.join(mapping.get(char, UNK) for char in text)
    return text_decoded

def get_ordered_chars(text):
    return [char for char, _ in Counter(text).most_common()]

def perform_mcmc(train_part, test_encoded, iters, print_n):
    best_score = -np.inf
    best_mapping = None
    train_chars_list = get_ordered_chars(train_part)
    test_chars_list = get_ordered_chars(test_encoded)

    train_bigrams = get_bigrams(train_part)
    train_bigrams_cnt = Counter(train_bigrams)

    mapping = dict(zip(test_chars_list, train_chars_list))
    text_decoded = decode_text_by_mapping(test_encoded, mapping)
    score = calc_log_likelihood(text_decoded, train_bigrams_cnt)
    trained_chars = train_chars_list.copy()

    for i in tqdm(range(iters)):
        trained_chars_new = random_swap_chars(trained_chars)
        mapping_new = dict(zip(test_chars_list, trained_chars_new))
        text_decoded_new = decode_text_by_mapping(test_encoded, mapping_new)
        score_new = calc_log_likelihood(text_decoded_new, train_bigrams_cnt)

        score_diff = np.exp(score_new - score)
        if score_diff > np.random.uniform(0, 1):
            score = score_new
            mapping = mapping_new
            trained_chars = trained_chars_new.copy()

        if score > best_score:
            best_score = score
            best_mapping = mapping

        if i % print_n == 0:
            print(f'Iter: {i}, best_score: {best_score}')
            res = decode_text_by_mapping(test_encoded, best_mapping)
            print(res[:300])

    return best_mapping

In [34]:
%%time

iterations = 10000
print_interval = 1000
mcmc_mapping = perform_mcmc(train_wap, test_encoded, iterations, print_interval)

  0%|          | 0/10000 [00:00<?, ?it/s]

Iter: 0, best_score: -66850.31549431116
двниптесз сшетесз лроиптесз иаскотзко жегов н оуялз ослеиовнтесз уройлн юлн сло жегов н иевариоа суесаи урослоялз аэа два мнипль н уогнб иевариоа дпмет кешдьй дотохов слоявжнй в сарадниа лотуь рвеиптся к крец утолниь сбнв с иог двпх сотдел н сбашет ие скотзыкнй тад уокрьвжнй урпд своречнвей ыекрнчет
Iter: 1000, best_score: -59207.835986021906
дсинулаяь ячалаяь тронулаяь неявольво шагос и опзть оятаносилаяь пройти эти ято шагос и насерное япаяен проятозть еще дсе минуты и погиб насерное думал вачдый долофос ятозсший с яередине толпы рсануляз в враю плотины ябис я ног дсуф яолдат и ябечал на явольжвий лед поврысший пруд ясоракисай жаврикал
Iter: 2000, best_score: -55004.2988246911
двинулась сжалась тронулась несколько шагов и опять остановилась пройти эти сто шагов и наверное спасен простоять еще две минуты и погиб наверное думал каждый долохов стоявший в середине толпы рванулся к краю плотины сбив с ног двух солдат и сбежал на скользкий лед покрыв

In [37]:
test_decoded_mcmc = decode_text_by_mapping(test_encoded, mcmc_mapping)
print(f"Точность: {get_accuracy_(test_encoded, test_decoded_mcmc, decode_mapping)}")

Точность: 1.0


In [38]:
test_decoded_mcmc[:500]

'двинулась сжалась тронулась несколько шагов и опять остановилась пройти эти сто шагов и наверное спасен простоять еще две минуты и погиб наверное думал каждый долохов стоявший в середине толпы рванулся к краю плотины сбив с ног двух солдат и сбежал на скользкий лед покрывший пруд сворачивай закричал он подпрыгивая по льду который трещал под ним сворачивай кричал он на орудие держит лед держал его но гнулся и трещал и очевидно было что не только под орудием или толпой народа но под ним одним он с'

### 4.	Расшифруйте сообщение:<br>
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
<br><br>
Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):<br>
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ
<br><br>

In [27]:
text_encoded = 'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

In [28]:
with open(PATH_WAR, 'r') as f_ru_1, open(PATH_ANNA, 'r') as f_ru_2:
    war_and_peace = f_ru_1.read()
    anna_karenina = f_ru_2.read()
    corpus_rus = war_and_peace + ' ' + anna_karenina

corpus_rus = preprocess_text(corpus_rus)

In [29]:
%%time
iterations = 500000
print_interval = 100000
mcmc_mapping = perform_mcmc(corpus_rus, text_encoded, iterations, print_interval)

  0%|          | 0/500000 [00:00<?, ?it/s]

Iter: 0, best_score: -1627.765474866931
олна рд раяасо иевьтнуидг ана пемса иевьтнуидг соклс б шсеые леезжоиач кесевдг ноыке пвемастсу лкевоо рлоые рд рло ляонтна пвтрануие а пенбмасо ьтклаьтнуидг зтнн йт пелнояиоо мосровсео йтятиао кбвлт хесч кеиомие ч иамоые ио езожтю
Iter: 100000, best_score: -1241.5561722197178
если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж
Iter: 200000, best_score: -1241.5561722197178
если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж
Iter: 300000, best_score: -1241.5561722197178
если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правил

In [30]:
text_encoded

'დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ'

In [31]:
text_decoded_mcmc = decode_text_by_mapping(text_encoded, mcmc_mapping)
text_decoded_mcmc

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'

### 6. Бонус

- **Криптография:** Эти модели можно использовать для расшифровки закодированных сообщений

- **NLP:** Исправление орфографических ошибок; определние языка (русский, английский или другой) - для разных языков будет разная частотность, анализ больших текстов: выявление закономерностей и тенденций

- **Сжатие данных:** например, алгоритм BPE

- **Машинное обучение:** Т к у разных текстов разная частотность n-грамм, то можно сопоставлять и находить похожие тексты по этим частотностям (если counter-ы похожи, то тексты одинаковые) - тематическое моделирование